# Predict Missing Markers Trajectories

**References:**
- Federolf PA (2013) *A Novel Approach to Solve the “Missing Marker Problem” in Marker-Based Motion Analysis That Exploits the Segment Coordination Patterns in Multi-Limb Motion Data.* PLOS ONE 8(10): e78689. <a href="https://doi.org/10.1371/journal.pone.0078689">https://doi.org/10.1371/journal.pone.0078689</a>
- Gløersen Ø, Federolf P (2016) *Predicting Missing Marker Trajectories in Human Motion Data Using Marker Intercorrelations.* PLOS ONE 11(3): e0152616. <a href="https://doi.org/10.1371/journal.pone.0152616">https://doi.org/10.1371/journal.pone.0152616</a>

**Author:** Robbin Romijnders

Marker-based motion analysis generally suffers from loss of marker information, for example due to occlusion or marker detachment. Naive approaches to fillings gaps in the marker trajectories are based on linear or spline interpolation, however these are mainly suitable for gaps of short duration. In **Federolf (2013)** and **Gløersen and Federolf (2016)** a method was proposed that takes advantage of intercorrelations between neighboring markers. The MATLAB source code is available as the Supporting Information online:  
<a href="https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0152616#sec018">https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0152616#sec018</a>.

Here, we have implemented the methods in Python for further use.

## Prerequisites

In [9]:
# Load libraries
from lib.utils import _load_file
from lib.preprocessing import _predict_missing_markers
import os, re
import numpy as np

from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [10]:
def mean_eucl_distance(y, yhat):
    """Computes the mean Euclidean distance between the original and estimated marker trajectories.

    Parameters
    ----------
    y : (N', 3) array_like
        The original data for which there are gaps.
    yhat : (N', 3) array_like
        The reconstructed data, for the same time steps.
    
    Returns
    -------
    _ : float
        The mean Euclidean distance.
    """
    
    # Compute the element-wise differences
    d = ( yhat - y )

    # Take the square of differences
    sqd = d**2

    # Sum the squared differences for each time step (i.e., each row)
    ssqd = np.sum(sqd, axis=1)

    # Take the square root of the sum of squared differences
    eucl_dist = np.sqrt(ssqd)
    return np.mean(eucl_dist)

In [11]:
# Load data
PARENT_FOLDER = "./data/test"  # set parent folder
data_full = _load_file(os.path.join(PARENT_FOLDER, "WalkL.mat"))  # load original gap-free data

In [12]:
# Get a list of filenames
filenames = [fname for fname in os.listdir(PARENT_FOLDER) if re.match("WalkL_[0-9]{4}.mat", fname)]

## Processing

In [13]:
# Get shape of data
n_time_steps, n_channels = data_full.shape
n_markers = n_channels // 3
print(f"Number of time steps: {n_time_steps}")
print(f"Number of markers: {n_markers}")

Number of time steps: 4300
Number of markers: 37


In [14]:
# Initialize dictionary to store results
evals = {}
for m in range(n_markers):
    evals[m] = {"files": [], "scores": [], "length_of_gaps": []}

In [15]:
# Loop over the filenames
for i, filename in enumerate(filenames[17:18]):
    # Print to user screen
    print(f"{i:2d}: {filename:s}")
    
    # -- Get the data --
    data_gaps = _load_file(os.path.join(PARENT_FOLDER, filename))

    # -- Reconstruct marker trajectories --
    if filename == "WalkL_0086.mat":
        continue
    data_filled = _predict_missing_markers(data_gaps, method="R1")

    # -- Evaluate --
    ix_channels_with_gaps, = np.nonzero(np.any(np.isnan(data_gaps), axis=0))
    ix_time_steps_with_gaps, = np.nonzero(np.any(np.isnan(data_gaps), axis=1))
    ix_markers_with_gaps = (ix_channels_with_gaps[2::3]//3)
    for ix_marker in ix_markers_with_gaps:
        # Find time steps with missing data
        ix_nans = np.argwhere(np.isnan(data_gaps[:,ix_marker*3]))[:,0]

        # Get the mean Euclidean distance
        med = mean_eucl_distance(data_full[ix_nans, 3*ix_marker:3*ix_marker+3], data_filled[ix_nans, 3*ix_marker:3*ix_marker+3])

        # Save to dictionary
        evals[ix_marker]["files"].append(filename)
        evals[ix_marker]["scores"].append(med)
        evals[ix_marker]["length_of_gaps"].append(len(ix_nans))


 0: WalkL_0086.mat


IndexError: index 0 is out of bounds for axis 0 with size 0

In [8]:
evals

{0: {'files': [], 'scores': [], 'length_of_gaps': []},
 1: {'files': [], 'scores': [], 'length_of_gaps': []},
 2: {'files': ['WalkL_0029.mat'],
  'scores': [10.217189959754496],
  'length_of_gaps': [368]},
 3: {'files': ['WalkL_0029.mat'],
  'scores': [14.8667426746506],
  'length_of_gaps': [351]},
 4: {'files': ['WalkL_0029.mat'],
  'scores': [13.146171667626243],
  'length_of_gaps': [72]},
 5: {'files': ['WalkL_0029.mat'],
  'scores': [15.01124053219431],
  'length_of_gaps': [92]},
 6: {'files': ['WalkL_0029.mat'],
  'scores': [66.14394112946732],
  'length_of_gaps': [274]},
 7: {'files': ['WalkL_0029.mat'],
  'scores': [21.554470362561574],
  'length_of_gaps': [144]},
 8: {'files': ['WalkL_0029.mat'],
  'scores': [44.570710288994064],
  'length_of_gaps': [171]},
 9: {'files': ['WalkL_0029.mat'],
  'scores': [23.122549859198415],
  'length_of_gaps': [426]},
 10: {'files': ['WalkL_0029.mat'],
  'scores': [53.00369745398529],
  'length_of_gaps': [43]},
 11: {'files': ['WalkL_0029.mat']

## Evaluation

To evaluate the methods of filling gaps in the marker trajectories, the mean Euclidean distances between the original gap-free and the reconstructed data were computed:
$\begin{align}
d^{(m)}_{i} &= \sqrt{(\hat{x}^{(m)}_{i}-x^{(m)}_{i})^{2} + (\hat{y}^{(m)}_{i}-y^{(m)}_{i})^{2} + (\hat{z}^{(m)}_{i}-z^{(m)}_{i})^{2}}, \\
\bar{d}^{(m)} &= \frac{1}{N'} \sum\limits_{i=1}^{N'} d^{(m)}_{i}
\end{align}$
with the superscript, $^{(m)}$, denoting the $m$-th marker, the subscript, $_{i}$, denoting the $i$-th time steps (or frame), $x$, $y$, and $z$ denoting the original marker trajectories in the corresponding directions, and their hatted equivalents, $\hat{x}$, $\hat{y}$, and $\hat{z}$ denoting the estimated (i.e. reconstructed) trajectories after interpolation, and $N'$ the total number of time steps with missing data for the current (i.e., $m$-th) marker.

In [18]:
for ix_marker in ix_markers_with_gaps:

    # Find time steps with missing data
    ix_nans = np.argwhere(np.isnan(data_gaps[:,ix_marker*3]))[:,0]

    # Differences
    med = mean_eucl_distance(data_full[ix_nans, 3*ix_marker:3*ix_marker+3], data_filled[ix_nans, 3*ix_marker:3*ix_marker+3])
    print(f"{ix_marker:2d}: {med:.0f}")

 0: 14
 1: 6
 2: 8
 9: 9
10: 11
13: 3
14: 2
17: 2
19: 4
20: 3
22: 2
23: 4
28: 1
32: 4
33: 1


In [10]:
# Plot trajectories for a given marker
fig = make_subplots(rows=3, cols=1, shared_xaxes=True)

for ix in ix_markers_with_gaps[:1]:
    # X coordinate
    fig.add_trace(go.Scatter(x=np.arange(data_full.shape[0]), y=data_full[:,ix*3], mode="lines", line=dict(color="rgba(0,0,0,0.2)", width=3)), row=1, col=1)
    fig.add_trace(go.Scatter(x=np.arange(data_gaps.shape[0]), y=data_gaps[:,ix*3], mode="lines", line=dict(color="rgba(0,0,0)", width=1)), row=1, col=1)
    fig.add_trace(go.Scatter(x=np.arange(data_filled.shape[0])[ix_nans], y=data_filled[ix_nans,ix*3], mode="lines", line=dict(color="rgba(255,0,0,1)", width=1, dash="dashdot")), row=1, col=1)

    # Y coordinate
    fig.add_trace(go.Scatter(x=np.arange(data_full.shape[0]), y=data_full[:,ix*3+1], mode="lines", line=dict(color="rgba(0,0,0,0.2)", width=3)), row=2, col=1)
    fig.add_trace(go.Scatter(x=np.arange(data_gaps.shape[0]), y=data_gaps[:,ix*3+1], mode="lines", line=dict(color="rgba(0,0,0)", width=1)), row=2, col=1)
    fig.add_trace(go.Scatter(x=np.arange(data_filled.shape[0])[ix_nans], y=data_filled[ix_nans,ix*3+1], mode="lines", line=dict(color="rgba(255,0,0,1)", width=1, dash="dashdot")), row=2, col=1)

    # Z coordinate
    fig.add_trace(go.Scatter(x=np.arange(data_full.shape[0]), y=data_full[:,ix*3+2], mode="lines", line=dict(color="rgba(0,0,0,0.2)", width=3)), row=3, col=1)
    fig.add_trace(go.Scatter(x=np.arange(data_gaps.shape[0]), y=data_gaps[:,ix*3+2], mode="lines", line=dict(color="rgba(0,0,0)", width=1)), row=3, col=1)
    fig.add_trace(go.Scatter(x=np.arange(data_filled.shape[0])[ix_nans], y=data_filled[ix_nans,ix*3+2], mode="lines", line=dict(color="rgba(255,0,0,1)", width=1, dash="dashdot")), row=3, col=1)
fig.update_layout(showlegend=False, width=960, height=540)
fig.show()